In [1]:
import requests
from bs4 import BeautifulSoup
import re
from urllib.parse import urljoin
import warnings
warnings.filterwarnings("ignore")

def scrape_text(url):
    """Fetch and clean visible text from a webpage."""
    try:
        resp = requests.get(url, headers={"User-Agent": "Mozilla/5.0"}, timeout=10)
        resp.raise_for_status()
    except Exception as e:
        print(f"Error fetching {url}: {e}")
        return ""

    soup = BeautifulSoup(resp.text, "html.parser")
    for tag in soup(["script", "style", "noscript"]):
        tag.decompose()

    text = soup.get_text(separator=" ", strip=True)
    text = re.sub(r"\s+", " ", text)
    return text

def find_relevant_pages(base_url, keywords):
    """Find pages by keyword relevance."""
    try:
        resp = requests.get(base_url, headers={"User-Agent": "Mozilla/5.0"}, timeout=10)
        resp.raise_for_status()
    except Exception as e:
        print(f"Error fetching {base_url}: {e}")
        return [base_url]

    soup = BeautifulSoup(resp.text, "html.parser")
    links = [a.get("href") for a in soup.find_all("a", href=True)]

    relevant_pages = []
    for link in links:
        if any(k in link.lower() for k in keywords):
            full_link = urljoin(base_url, link)
            if full_link not in relevant_pages:
                relevant_pages.append(full_link)

    return relevant_pages

def summarize_text(text, max_sentences=3):
    """Generate a short, crisp company summary."""
    sentences = re.split(r'(?<=[.!?]) +', text)
    keywords = ["about", "vision", "mission", "objective", "focus", "establish",
                "function", "research", "development", "sustain", "industry"]

    scored = []
    for s in sentences:
        score = sum(k in s.lower() for k in keywords)
        if 40 < len(s) < 250:  # avoid junk sentences
            scored.append((score, s.strip()))

    scored.sort(key=lambda x: (-x[0], len(x[1])))
    top_sentences = [s for _, s in scored[:max_sentences]]
    return " ".join(top_sentences)

def extract_contacts(text):
    """Extract emails, phone numbers, and possible addresses."""
    emails = list(set(re.findall(r"[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}", text)))
    phones = list(set(re.findall(r"(?:\+?\d{1,3}[-.\s]?)?\(?\d{2,4}\)?[-.\s]?\d{3,5}[-.\s]?\d{3,5}", text)))

    # crude heuristic for address
    address_candidates = re.findall(r"([A-Z][^,]+(?:Street|Strasse|Road|Rd|Avenue|Ave|Lane|Ln|Boulevard|Blvd|Way|Place|Pl|Square|Sq|Drive|Dr|Building|Bldg|No\.|Kaiserstraße).+)", text, re.IGNORECASE)
    address = address_candidates[0] if address_candidates else None

    return {
        "emails": emails,
        "phones": phones,
        "address": address
    }

def company_profile(base_url):
    """Scrape summary + contacts separately for better results."""
    # Pages for summary
    summary_pages = [base_url] + find_relevant_pages(base_url, ["about", "vision", "mission", "who-we-are", "company", "corporate"])
    
    # Pages for contact info
    contact_pages = find_relevant_pages(base_url, ["contact", "impressum", "imprint"])
    if not contact_pages:
        contact_pages = [base_url]  # fallback

    # Build summary
    summary_text = ""
    for page in summary_pages[:5]:
        summary_text += " " + scrape_text(page)
    summary = summarize_text(summary_text)

    # Extract contact info
    contact_text = ""
    for page in contact_pages[:3]:
        contact_text += " " + scrape_text(page)
    contacts = extract_contacts(contact_text)

    return {
        "url": base_url,
        "summary": summary,
        "contacts": contacts
    }

# Example usage
if __name__ == "__main__":
    urls = ["https://bathcanalcraft.co.uk","https://www.kit.edu/","https://mpob.gov.my/"] 
    for url in urls:
        print(f"\n---Company {url} Summary ---\n")
        profile = company_profile(url)
        print("Website:", profile["url"])
        print("\nSummary:", profile["summary"])
        print("\nEmails:", profile["contacts"]["emails"])
        print("Phones:", profile["contacts"]["phones"])
        print("Address:", profile["contacts"]["address"])



---Company https://bathcanalcraft.co.uk Summary ---

Website: https://bathcanalcraft.co.uk

Summary: From the initial concept to the maiden voyage, we prioritise your vision every step of the way. Are you looking for a beautifully formed narrowboat, uniquely designed to fit your lifestyle? Join us in redefining the future of narrowboat cruising, where tradition meets innovation in harmony with the environment.

Emails: ['bathcanalcraft@icloud.com']
Phones: ['+44 7538 784 613']
Address: Contact - Bath Canal Craft ‍ ‍ Home Boats All Boats NB ‘Alica-Lee‘ (in planning) NB ‘Fram‘ NB ‘Silly Knot To‘ NB ‘Slapdash Lilly‘ NB ‘The Doran‘ (sailaway) NB ‘Elaine‘ NB ‘Velore‘ Contact If you’d like to discuss a personalised narrowboat build then please feel free call, email, or come visit… +44 7538 784 613 bathcanalcraft@icloud.com Bath Canal Craft Ltd. Deverill Storage Longbridge Deverill Warminster Wiltshire BA12 7FB Bath Canal Craft Ltd. | All Rights Reserved © 2025

---Company https://www.kit.ed

***Smart summary about company using facebook HuggingFace model.***

In [ ]:
import re
import requests
from bs4 import BeautifulSoup
from transformers import pipeline
import warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", message=".*max_new_tokens.*")
warnings.filterwarnings("ignore", category=UserWarning)

# Optional JS rendering
try:
    from requests_html import HTMLSession
    JS_RENDER_AVAILABLE = True
except ImportError:
    JS_RENDER_AVAILABLE = False

# Load local summarizer
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
# summarizer = pipeline("summarization", model="t5-small")
# -------------------------------
# 1. Scrape Website Text
# -------------------------------
def scrape_website(url):
    text = ""
    try:
        r = requests.get(url, headers={"User-Agent": "Mozilla/5.0"}, timeout=10)
        soup = BeautifulSoup(r.text, "html.parser")
        for script in soup(["script", "style", "noscript"]):
            script.extract()
        text = " ".join(soup.stripped_strings)
    except Exception as e:
        print(f"Normal request failed: {e}")

    # Fallback to JS-rendered
    if len(text.split()) < 50 and JS_RENDER_AVAILABLE:
        try:
            session = HTMLSession()
            r = session.get(url)
            r.html.render(timeout=20)
            text = r.html.text
        except Exception as e:
            print(f"JS render failed: {e}")

    return text

# -------------------------------
# 3. Extract Objectives and Services
# -------------------------------
def extract_objective_service_text(text):
    paragraphs = [p.strip() for p in text.split("\n") if len(p.strip()) > 40]

    # Keywords
    objective_keywords = ["mission", "objective", "goal", "vision", "purpose", "focus", "aim"]
    service_keywords = ["service", "product", "offer", "specialize", "provide", "solutions", "design", "build", "manufacture"]

    # Extract paragraphs
    objective_paragraphs = [p for p in paragraphs if any(k in p.lower() for k in objective_keywords)]
    service_paragraphs = [p for p in paragraphs if any(k in p.lower() for k in service_keywords)]

    combined = objective_paragraphs + service_paragraphs
    if combined:
        return " ".join(combined)
    else:
        # fallback to first few paragraphs
        return " ".join(paragraphs[:10])

# -------------------------------
# 4. Generate Crisp Summary
# -------------------------------
def generate_summary(text, company_name="The company"):
    text = text[:2000]  # HuggingFace token limit
    input_length = len(text.split())
    max_length = min(120, input_length)  # Ensure max_length <= input_length
    min_length = min(40, max_length - 1) if max_length > 40 else 10
    result = summarizer(text, max_length=max_length, min_length=min_length, do_sample=False)
    return f"**{company_name}**: {result[0]['summary_text']}"

# -------------------------------
# 5. Full Company Analyzer
# -------------------------------
def     analyze_company(url, company_name="The company"):
    text = scrape_website(url)
    objective_service_text = extract_objective_service_text(text)
    if objective_service_text:
        summary = generate_summary(objective_service_text, company_name)
    else:
        summary = "None"
    
    return summary
    

# -------------------------------
# Example Usage
# -------------------------------
if __name__ == "__main__":
    websites = ['https://www.bathcanalcraft.co.uk',
        "https://www.kit.edu/", 
        "https://www.mpob.gov.my/",
        "https://www.nseindia.com/", 'https://www.research.gla.ac.uk', 
        'https://www.bebob.de', 'https://www.as.gov.qa', 'https://www.airjouletech.com',
        'https://www.moser-konstruktion.de', 'https://www.ac.sce.ac.il'
    ]

    for url in websites:
        print(f"\n--- Scraping {url} ---\n")
        summary = analyze_company(url)
        print(summary)


Device set to use cpu



--- Scraping https://www.bathcanalcraft.co.uk ---

Summary:
 **https://www.bathcanalcraft.co.uk**: Bath Canal Craft offers a truly bespoke service, working closely with you to ensure you have control over the design decisions that matter most. Join us in redefining the future of narrowboat cruising, where tradition meets innovation in harmony with the environment.

--- Scraping https://www.kit.edu/ ---

Summary:
 **https://www.kit.edu/**: KIT - Karlsruher Institut für Technologie Navigation überspringen Home Leichte Sprache Gebärdensprache Impressum Datenschutz Barrierefreiheit Sitemap Intranet KIT en suchen suchen.

--- Scraping https://www.mpob.gov.my/ ---

Summary:
 **https://www.mpob.gov.my/**: Malaysian Palm Oil Board – 6, Persiaran Institusi, Bandar Baru Bangi 43000 Kajang Selangor, Malaysia CERTIFIED TO ISO 9001:2015 CERT. NO.: QMS 02602 Contact Us Sitemap Menu Corporate Info About Us MPOB Logo Vision & Mission Organisation Chart Clientsâ Charter Top Management Board Members 

In [4]:
import re
import requests
from bs4 import BeautifulSoup
from transformers import pipeline
from concurrent.futures import ThreadPoolExecutor

# Optional JS rendering
try:
    from requests_html import HTMLSession
    JS_RENDER_AVAILABLE = True
except ImportError:
    JS_RENDER_AVAILABLE = False

# -------------------------------
# Load summarizer once
# -------------------------------
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
# summarizer = pipeline("summarization", model="t5-small")
# -------------------------------
# 1. Scrape Website Text
# -------------------------------
def scrape_website(url):
    """Fetch visible text from webpage with headers to avoid 403 errors."""
    text = ""
    try:
        r = requests.get(url, headers={"User-Agent": "Mozilla/5.0"}, timeout=10)
        r.raise_for_status()
        soup = BeautifulSoup(r.text, "html.parser")
        for tag in soup(["script", "style", "noscript"]):
            tag.decompose()
        text = " ".join(soup.stripped_strings)
    except Exception as e:
        print(f"Normal request failed for {url}: {e}")

    # Fallback to JS-rendered
    if len(text.split()) < 50 and JS_RENDER_AVAILABLE:
        try:
            session = HTMLSession()
            r = session.get(url)
            r.html.render(timeout=20)
            text = r.html.text
        except Exception as e:
            print(f"JS render failed for {url}: {e}")

    return text

# -------------------------------
# 2. Extract Objectives & Services
# -------------------------------
def extract_objective_service_text(text):
    paragraphs = [p.strip() for p in text.split("\n") if len(p.strip()) > 40]

    objective_keywords = ["mission", "objective", "goal", "vision", "purpose", "focus", "aim"]
    service_keywords = ["service", "product", "offer", "specialize", "provide", "solutions", "design", "build", "manufacture"]

    objective_paragraphs = [p for p in paragraphs if any(k in p.lower() for k in objective_keywords)]
    service_paragraphs = [p for p in paragraphs if any(k in p.lower() for k in service_keywords)]

    combined = objective_paragraphs + service_paragraphs
    if combined:
        return " ".join(combined)
    return " ".join(paragraphs[:10]) if paragraphs else ""

# -------------------------------
# 3. Generate Crisp Summary
# -------------------------------
def generate_summary(text, company_name="The company"):
    text = text[:2000]  # HuggingFace token limit
    input_length = len(text.split())
    max_length = min(120, input_length)
    min_length = min(40, max_length - 1) if max_length > 40 else 10
    result = summarizer(text, max_length=max_length, min_length=min_length, do_sample=False)
    return f"**{company_name}**: {result[0]['summary_text']}"

# -------------------------------
# 4. Full Company Analyzer
# -------------------------------
def analyze_company(url, company_name="The company"):
    text = scrape_website(url)
    objective_service_text = extract_objective_service_text(text)
    summary = generate_summary(objective_service_text, company_name) if objective_service_text else "None"
    return {
        "summary": summary
    }

# -------------------------------
# 5. Process Multiple Companies Efficiently
# -------------------------------
def analyze_multiple_companies(websites, max_workers=4):
    results = {}
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        future_to_company = {executor.submit(analyze_company, url, name): name for url, name in websites}
        for future in future_to_company:
            name = future_to_company[future]
            try:
                results[name] = future.result()
            except Exception as e:
                results[name] = {"summary": f"Error: {e}"}
    return results

# -------------------------------
# Example Usage
# -------------------------------
if __name__ == "__main__":
    websites = [
        ("https://bathcanalcraft.co.uk", "Bath Canal Craft"),
        ("https://www.kit.edu/", "Karlsruhe Institute of Technology"),
        ("https://mpob.gov.my/", "Malaysian Palm Oil Board"),
        ("https://www.nseindia.com/", "NSE India")
    ]

    results = analyze_multiple_companies(websites, max_workers=4)
    for name, data in results.items():
        print(f"\n--- {name} ---\n")
        print("Summary:\n", data["summary"])


Device set to use cpu



--- Bath Canal Craft ---

Summary:
 **Bath Canal Craft**: Bath Canal Craft offers a truly bespoke service, working closely with you to ensure you have control over the design decisions that matter most. Join us in redefining the future of narrowboat cruising, where tradition meets innovation in harmony with the environment.

--- Karlsruhe Institute of Technology ---

Summary:
 **Karlsruhe Institute of Technology**: KIT - Karlsruher Institut für Technologie Navigation überspringen Home Leichte Sprache Gebärdensprache Impressum Datenschutz Barrierefreiheit Sitemap Intranet KIT en suchen suchen.

--- Malaysian Palm Oil Board ---

Summary:
 **Malaysian Palm Oil Board**: Malaysian Palm Oil Board – 6, Persiaran Institusi, Bandar Baru Bangi 43000 Kajang Selangor, Malaysia CERTIFIED TO ISO 9001:2015 CERT. NO.: QMS 02602 Contact Us Sitemap Menu Corporate Info About Us MPOB Logo Vision & Mission Organisation Chart Clientsâ Charter Top Management Board Members Division Research & Development (

In [5]:
import re
import requests
from bs4 import BeautifulSoup

def normalize_phone(number: str) -> str:
    number = number.strip()
    number = re.sub(r"[^\d+]", "", number)  # keep only digits and '+'

    # Malaysia example
    if number.startswith("603"):
        return "+60 " + number[2:]
    if number.startswith("+60"):
        return number
    return number

def extract_address(text_lines):
    ignore_keywords = ["skip to content", "menu", "footer", "sitemap", "disclaimer", "policy"]
    address_keywords = r"(Persiaran|Bandar|Selangor|Kajang|Malaysia)"

    for i, line in enumerate(text_lines):
        line_lower = line.lower()
        if any(k in line_lower for k in ignore_keywords):
            continue  # skip UI/footer lines

        if re.search(r"\d+", line) and re.search(address_keywords, line, re.IGNORECASE):
            # include next line if short and not ignored
            candidate = line
            if i + 1 < len(text_lines):
                next_line = text_lines[i+1].strip()
                if len(next_line) < 80 and not any(k in next_line.lower() for k in ignore_keywords):
                    candidate += " " + next_line
            return candidate.strip()
    return None


def scrape_contact_info(url):
    try:
        resp = requests.get(url, headers={"User-Agent": "Mozilla/5.0"}, timeout=10)
        resp.raise_for_status()
    except Exception as e:
        print(f"Error fetching {url}: {e}")
        return {}

    soup = BeautifulSoup(resp.text, "html.parser")

    # Replace <br> with newline to preserve line separation
    for br in soup.find_all("br"):
        br.replace_with("\n")

    # Extract visible text lines
    text_lines = [line.strip() for line in soup.stripped_strings if line.strip()]

    # ----------------- Extract emails -----------------
    emails = set()
    # from <a href="mailto:...">
    for a in soup.find_all("a", href=True):
        if a['href'].lower().startswith("mailto:"):
            emails.add(a['href'].split(":", 1)[1].strip())
    # fallback: regex in text
    for line in text_lines:
        for e in re.findall(r"[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}", line):
            emails.add(e)

    # ----------------- Extract phones -----------------
    phones = set()
    for a in soup.find_all("a", href=True):
        if a['href'].lower().startswith("tel:"):
            phones.add(normalize_phone(a['href'].split(":", 1)[1].strip()))
    # fallback: regex in text
    phone_pattern = r"(?:Tel|Phone|Telefon)[:.]?\s*([+0-9\s\-]{6,})"
    for line in text_lines:
        match = re.search(phone_pattern, line, re.IGNORECASE)
        if match:
            phones.add(normalize_phone(match.group(1)))

    # ----------------- Extract fax -----------------
    fax_numbers = set()
    fax_pattern = r"(?:Fax)[:.]?\s*([+0-9\s\-]{6,})"
    for line in text_lines:
        match = re.search(fax_pattern, line, re.IGNORECASE)
        if match:
            fax_numbers.add(normalize_phone(match.group(1)))

    # ----------------- Extract address -----------------
    address_keywords = r"(Persiaran|Bandar|Selangor|Kajang|Malaysia)"
    # Replace the previous address parsing with:
    address = extract_address(text_lines)

    return {
        "emails": list(emails),
        "phones": list(phones),
        "fax": list(fax_numbers),
        "address": address
    }

# -------------------------------
# Example
# -------------------------------
if __name__ == "__main__":
    websites = ['https://www.bathcanalcraft.co.uk',
        "https://www.kit.edu/", 
        "https://www.mpob.gov.my/",
        "https://www.nseindia.com/", 'https://www.research.gla.ac.uk', 
        'https://www.bebob.de', 'https://www.as.gov.qa', 'https://www.airjouletech.com',
        'https://www.moser-konstruktion.de', 'https://www.ac.sce.ac.il'
    ]

    for url in websites:
        print(f"\n--- Scraping {url} ---\n")
        summary = scrape_contact_info(url)
        print(summary)


--- Scraping https://www.bathcanalcraft.co.uk ---

{'emails': [], 'phones': [], 'fax': [], 'address': None}

--- Scraping https://www.kit.edu/ ---

{'emails': [], 'phones': [], 'fax': [], 'address': None}

--- Scraping https://www.mpob.gov.my/ ---

{'emails': ['general@mpob.gov.my'], 'phones': ['+60 387694400'], 'fax': ['+60 389259446'], 'address': 'Malaysian Palm Oil Board – 6, Persiaran Institusi, Bandar Baru Bangi<br>43000 Kajang Selangor, Malaysia'}

--- Scraping https://www.nseindia.com/ ---

{'emails': [], 'phones': [], 'fax': [], 'address': None}

--- Scraping https://www.research.gla.ac.uk ---

Error fetching https://www.research.gla.ac.uk: HTTPSConnectionPool(host='www.research.gla.ac.uk', port=443): Max retries exceeded with url: / (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x00000170F867B250>: Failed to resolve 'www.research.gla.ac.uk' ([Errno 11001] getaddrinfo failed)"))
{}

--- Scraping https://www.bebob.de ---

Error fetching https://w

In [ ]:
# Ensure playwright is installed
%pip install playwright
# If running for the first time, uncomment the next line to install browser binaries:
# !playwright install

import re
import asyncio
from playwright.async_api import async_playwright
from bs4 import BeautifulSoup

# Use existing variables from the notebook:
# ADDRESS_KEYWORDS, PHONE_PATTERN, FAX_PATTERN, EMAIL_PATTERN, IGNORE_KEYWORDS, websites

async def scrape_contact_info(url: str):
    emails, phones, fax_numbers, address = [], [], [], None
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()
        await page.goto(url, timeout=30000)
        content = await page.content()  # rendered HTML
        await browser.close()

    soup = BeautifulSoup(content, "html.parser")

    # Emails from mailto links
    for a in soup.find_all("a", href=True):
        if a['href'].startswith("mailto:"):
            emails.append(a['href'].split(":", 1)[1].strip())

    # Emails from text fallback
    for line in soup.stripped_strings:
        for e in re.findall(EMAIL_PATTERN, line):
            emails.append(e.replace(" ", ""))

    # Phones
    for line in soup.stripped_strings:
        match = re.search(PHONE_PATTERN, line, re.IGNORECASE)
        if match:
            phones.append(match.group(1).strip())

    # Fax
    for line in soup.stripped_strings:
        match = re.search(FAX_PATTERN, line, re.IGNORECASE)
        if match:
            fax_numbers.append(match.group(1).strip())

    # Address (first plausible line)
    for line in soup.stripped_strings:
        if any(k in line.lower() for k in IGNORE_KEYWORDS):
            continue
        if re.search(r"\d+", line) and re.search(ADDRESS_KEYWORDS, line, re.IGNORECASE):
            address = line.strip()
            break

    return {
        "emails": emails,
        "phones": phones,
        "fax": fax_numbers,
        "address": address
    }

# Run for all websites in the global 'websites' variable
websites = ['https://www.bathcanalcraft.co.uk',
        "https://www.kit.edu/", 
        "https://www.mpob.gov.my/",
        "https://www.nseindia.com/", 'https://www.research.gla.ac.uk', 
        'https://www.bebob.de', 'https://www.as.gov.qa', 'https://www.airjouletech.com',
        'https://www.moser-konstruktion.de', 'https://www.ac.sce.ac.il'
    ]

results = await asyncio.gather(*[scrape_contact_info(url) for url in websites])
for url, data in zip(websites, results):
    print(f"\n--- {url} ---")
    print("Address:", data["address"])
    print("Phones:", data["phones"])
    print("Emails:", data["emails"])
    print("Fax:", data["fax"])


Note: you may need to restart the kernel to use updated packages.


NameError: name 'websites' is not defined